In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
data_dir = '/content/drive/MyDrive/Dataset/ds2'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'validation']}

In [7]:
from torch.utils.data import dataset
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'validation']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
class_names = image_datasets['train'].classes
print(class_names)
print(dataset_sizes)

['clean', 'water']
{'train': 700, 'validation': 150}


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [8]:
import timm
import torch
import torch.nn as nn

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet18 and change the classifier for 10 classes
model_tl = timm.create_model(
    'resnet18',
    pretrained=True,   # Use ImageNet weights
    num_classes=10     # Change final layer to match dataset
)

# Freeze all layers except the final classifier
for param in model_tl.parameters():
    param.requires_grad = False

for param in model_tl.fc.parameters():  # Only last fc layer trainable
    param.requires_grad = True

# Move to device
model_tl = model_tl.to(device)

# Optional: print summary
from torchsummary import summary
summary(model_tl, input_size=(3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
          Identity-7           [-1, 64, 56, 56]               0
              ReLU-8           [-1, 64, 56, 56]               0
          Identity-9           [-1, 64, 56, 56]               0
           Conv2d-10           [-1, 64, 56, 56]          36,864
      BatchNorm2d-11           [-1, 64, 56, 56]             128
             ReLU-12           [-1, 64, 56, 56]               0
       BasicBlock-13           [-1, 64, 56, 56]               0
           Conv2d-14           [-1, 64,

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_tl.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    for phase in ['train', 'validation']:
        if phase == 'train':
            model_tl.train()
        else:
            model_tl.eval()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
            outputs = model_tl(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            if phase == 'train':
                loss.backward()
                optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects.double() / dataset_sizes[phase]

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        phase, epoch_loss, epoch_acc))
print('Training complete')

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


validation Loss: 2.3906 Acc: 0.0000
validation Loss: 2.3906 Acc: 0.0000
validation Loss: 2.3906 Acc: 0.0000
validation Loss: 2.3906 Acc: 0.0000
validation Loss: 2.3906 Acc: 0.0000
validation Loss: 2.3906 Acc: 0.0000
validation Loss: 2.3906 Acc: 0.0000
validation Loss: 2.3906 Acc: 0.0000
validation Loss: 2.3906 Acc: 0.0000


In [ ]:
torch.save(model_tl.state_dict(), 'vgg16_tl.pth')

In [ ]:
# model = models.vgg16()  # Initialize the model
# model.load_state_dict(torch.load('./vgg16_cifar10_without_transferlearning.pth'))
# model = model.to(device)


from torchvision import models

model = models.vgg16(pretrained=False)  # Initialize VGG16 without pretrained weights

# Modify the classifier to match the saved model
num_features = model.classifier[6].in_features
features = list(model.classifier.children())[:-1]  # Remove last layer
features.extend([nn.Linear(num_features, 10)])  # Adapt to your dataset, CIFAR10 has 10 classes
model.classifier = nn.Sequential(*features)

# Load the state dict (model weights)
model.load_state_dict(torch.load('./vgg16_tl.pth'))

# Move the model to the appropriate device (GPU or CPU)
model = model.to(device)

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# 1. Load the image
image_path = '/content/drive/MyDrive/Dataset/ds2/test/clean/IMG_1085.JPG'  # Change this to the path of your image
image = Image.open(image_path)

# 2. Preprocess the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize it
])
input_image = transform(image).unsqueeze(0)  # Add a batch dimension

# 3. Put the model in evaluation mode
model.eval()

# 4. Make a prediction
with torch.no_grad():
    # Move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_image = input_image.to('cuda')
        model.to('cuda')

    output = model(input_image)

# 5. Interpret the output
_, predicted = torch.max(output, 1)  # Get the index of the highest log-probability

# Get the class label (modify this part according to your dataset classes)
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
  # Replace with your own class labels
predicted_class = class_labels[predicted.item()]

print("Predicted Class: ", predicted_class)